In [17]:
import torch 
import pandas as pd
import numpy as np

In [2]:
def training(model , train_df , criterion , optimizer , device ):
    train_loss = 0
    model.train()
    for X , y in train_df:
        X ,y = X.to(device) , y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred.squeeze(1) , y)
        train_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_df)
     
    return train_loss

In [3]:
def testing(model , test_df , criterion , device):
    test_loss = 0
    true_labels = []
    pred_labels = []
    model.eval()
    with torch.inference_mode():
        
        for X ,y in test_df:
            X ,y = X.to(device) , y.to(device)
            y_pred = model(X)
            
            pred_labels += y_pred.tolist()
            true_labels += y.tolist()
            
            loss = criterion(y_pred.squeeze(1) , y)
            test_loss += loss
        test_loss /= len(test_df)
    
    return test_loss , true_labels , pred_labels

In [28]:
def submit(model , test_loader , ids , device):
    preds = []
    model.eval()
    with torch.inference_mode():
        for X in test_loader:
            X = X[0].to(device)
            y_pred = model(X).squeeze(1).float()
            preds += y_pred.tolist()
    
    #print(np.array(preds))
    df = pd.DataFrame({
         
        "ID_code" : ids.values , 
        "target" : np.array(preds)
        
    })
    
    df.to_csv("submission3.csv" , index = False)